In [ ]:
from client import create_genai_client

client = create_genai_client()

In [ ]:
available_models = client.models.list()
model_names = [model.name for model in available_models]

In [ ]:
import ipywidgets as widgets
from IPython.display import display

model_selector = widgets.Dropdown(
    options=model_names,
    description="Model:",
)

display(model_selector)


def on_model_change(change):
    global MODEL_ID
    MODEL_ID = change.new
    print(f"Selected model: {MODEL_ID}")


model_selector.observe(on_model_change, names="value")

In [ ]:
import time

from google.genai import types
from IPython.display import Markdown

time.sleep(5)

config = types.GenerateContentConfig(
    system_instruction=types.Part.from_text(
        text="You are a helpful and knowledgeable AI assistant. Answer questions clearly, accurately, and provide additional context when relevant."
    )
)

contents = types.Part.from_text(text="Hello, what model are you?")

response = client.models.generate_content(
    model=MODEL_ID,
    contents=contents,
    config=config,
)

Markdown(response.text)

In [ ]:
import time
from pathlib import Path

from google.genai import types
from IPython.display import Markdown
from utils import read_text_from_file

time.sleep(5)

project_root = Path("../../../").resolve()

system_prompt_file = project_root / "prompts" / "system" / "general-purpose.md"
try:
    system_prompt = read_text_from_file(system_prompt_file)
except FileNotFoundError:
    print("Error: The specified file was not found.")
except IOError as e:
    print(f"An error occurred while reading the file: {e}")

system_parts = [
    types.Part.from_text(text=system_prompt),
    types.Part.from_text(
        text="Answer questions clearly, accurately, and provide additional context when relevant."
    ),
]

config = types.GenerateContentConfig(
    candidate_count=1,
    frequency_penalty=0.0,
    presence_penalty=0.0,
    seed=5,
    stop_sequences=["STOP!"],
    system_instruction=system_parts,
    temperature=0.4,
    top_k=20,
    top_p=0.95,
)

user_prompt_file = project_root / "prompts" / "user" / "hello.md"
try:
    user_prompt = read_text_from_file(user_prompt_file)
except FileNotFoundError:
    print("Error: The specified file was not found.")
except IOError as e:
    print(f"An error occurred while reading the file: {e}")

user_parts = types.Part.from_text(text=user_prompt)

contents = user_parts

response = client.models.generate_content(
    config=config,
    contents=contents,
    model=MODEL_ID,
)

Markdown(response.text)